In [ ]:
# API token and CA crt

$serviceAccount = "cp-api-explorer"
# get right secret
$secretName = kubectl get serviceaccount cp-api-explorer -o json | ConvertFrom-Json `
    | % { $_.secrets } `
    | ? { $_.name.startsWith("$($serviceAccount)-token-") } `
    | select -First 1 | % { $_.name }
$secretName
# token from secretName
$token = kubectl get secret $secretName -o json | ConvertFrom-Json | % { $_.data.token | base64 -d }
# ca.crt
$caCrtFilename = "./k8sca.crt"
kubectl get secret $secretName -o json | ConvertFrom-Json | % { $_.data."ca.crt" | base64 -d } | Out-File $caCrtFilename
$caCrtFilename 
# API server URL
$apiServerUrl = kubectl config view -o json | jq -c '.clusters[] | select(.name == ""minikube"") | .cluster.server'
$apiServerUrl

cp-api-explorer-token-sbfpk
./k8sca.crt
"https://192.168.64.6:8443"


In [ ]:
# test - name of first pod in default NS
/usr/bin/curl -s "$apiServerUrl/api/v1/namespaces/default/pods" `
   --header "Authorization: Bearer $token" --cacert $caCrtFilename `
   | jq -r '.items[0] | .metadata.name'

middleware-service


In [ ]:
# list of namespaces
$namespaces = /usr/bin/curl -s "$apiServerUrl/api/v1/namespaces" `
   --header "Authorization: Bearer $token" --cacert $caCrtFilename `
   | jq -r '.items[] | .metadata.name'
$namespaces

default
kube-node-lease
kube-public
kube-system
lab


In [ ]:
# learn from masters - verbose mode of kubectl
kubectl get svc -v=8

I0331 16:31:11.700990    4870 loader.go:372] Config loaded from file:  /Users/mkoldov/.kube/config
I0331 16:31:11.703243    4870 cert_rotation.go:137] Starting client certificate rotation controller
I0331 16:31:11.710653    4870 round_trippers.go:463] GET https://192.168.64.6:8443/api/v1/namespaces/default/services?limit=500
I0331 16:31:11.710673    4870 round_trippers.go:469] Request Headers:
I0331 16:31:11.710688    4870 round_trippers.go:473]     Accept: application/json;as=Table;v=v1;g=meta.k8s.io,application/json;as=Table;v=v1beta1;g=meta.k8s.io,application/json
I0331 16:31:11.710696    4870 round_trippers.go:473]     User-Agent: kubectl/v1.23.3 (darwin/amd64) kubernetes/816c97a
I0331 16:31:11.721360    4870 round_trippers.go:574] Response Status: 200 OK in 10 milliseconds
I0331 16:31:11.721385    4870 round_trippers.go:577] Response Headers:
I0331 16:31:11.721404    4870 round_trippers.go:580]     Content-Type: application/json
I0331 16:31:11.721423    4870 round_trippers.go:580]

In [ ]:
# verbose - list services (expect to fail - no permissions to access SERVICES!)
/usr/bin/curl -s "$apiServerUrl/api/v1/namespaces/default/services" `
   --header "Authorization: Bearer $token" --cacert $caCrtFilename `


{
  "kind": "Status",
  "apiVersion": "v1",
  "metadata": {},
  "status": "Failure",
  "message": "services is forbidden: User \"system:serviceaccount:default:cp-api-explorer\" cannot list resource \"services\" in API group \"\" in the namespace \"default\"",
  "reason": "Forbidden",
  "details": {
    "kind": "services"
  },
  "code": 403
}


In [ ]:
# all pods per namespace with IP address
$namespaces | % {
    $namespace = $_
    /usr/bin/curl -s "$apiServerUrl/api/v1/namespaces/$namespace/pods" `
        --header "Authorization: Bearer $token" --cacert $caCrtFilename `
        | ConvertFrom-Json | % { $_.items } | % {
            [PSCustomObject] @{
                namespace = $namespace;
                name = $_.metadata.name;
                ip = $_.status.podIPs[0].ip
            }
        }
}


namespace   name                    ip
---------   ----                    --
default     middleware-service      172.17.0.13
default     web-1                   172.17.0.3
default     web-10                  172.17.0.12
default     web-2                   172.17.0.4
default     web-3                   172.17.0.5
default     web-4                   172.17.0.6
default     web-5                   172.17.0.7
default     web-6                   172.17.0.8
default     web-7                   172.17.0.9
default     web-8                   172.17.0.10
default     web-9                   172.17.0.11
kube-system coredns-64897985d-tb6lt 172.17.0.2
kube-system etcd-minikube           192.168.64…
kube-system kube-apiserver-minikube 192.168.64…
kube-system kube-controller-manage… 192.168.64…
kube-system kube-proxy-m694p        192.168.64…
kube-system kube-scheduler-minikube 192.168.64…
kube-system storage-provisioner     192.168.64…
lab         web-1                   172.17.0.14
lab         web-2

In [ ]:
# all pods per namespace with IP address
$objects = $namespaces | % {
    $namespace = $_
    $pods = /usr/bin/curl -s "$apiServerUrl/api/v1/namespaces/$namespace/pods" `
        --header "Authorization: Bearer $token" --cacert $caCrtFilename `
        | ConvertFrom-Json | % { $_.items } | % {
            [PSCustomObject] @{
                namespace = $namespace;
                name = $_.metadata.name;
                ip = $_.status.podIPs[0].ip
            }
        }
    $ips = $pods | % { $_.ip }
    if ($ips.Length -gt 0) { 
        # Write-Host $namespace ": " $ips 
        [PSCustomObject] @{
            name = "ns-$namespace-ips";
            id = "id-ns-$namespace-ips";
            description = "IP adresses in namespace $namespace";
            ranges = $ips
        }
    }
}

$objects


name               id                    description                          ranges
----               --                    -----------                          ------
ns-default-ips     id-ns-default-ips     IP adresses in namespace default     {172.17.0.13, 172.17…
ns-kube-system-ips id-ns-kube-system-ips IP adresses in namespace kube-system {172.17.0.2, 192.168…
ns-lab-ips         id-ns-lab-ips         IP adresses in namespace lab         {172.17.0.14, 172.17…



In [ ]:
[PSCustomObject] @{
        "version" = "1.0";
        "description" = "Generic Data Center file example";
        "objects" = $objects
} | ConvertTo-Json -Depth 10 | jq .

{
  "version": "1.0",
  "description": "Generic Data Center file example",
  "objects": [
    {
      "name": "ns-default-ips",
      "id": "id-ns-default-ips",
      "description": "IP adresses in namespace default",
      "ranges": [
        "172.17.0.13",
        "172.17.0.3",
        "172.17.0.12",
        "172.17.0.4",
        "172.17.0.5",
        "172.17.0.6",
        "172.17.0.7",
        "172.17.0.8",
        "172.17.0.9",
        "172.17.0.10",
        "172.17.0.11"
      ]
    },
    {
      "name": "ns-kube-system-ips",
      "id": "id-ns-kube-system-ips",
      "description": "IP adresses in namespace kube-system",
      "ranges": [
        "172.17.0.2",
        "192.168.64.6",
        "192.168.64.6",
        "192.168.64.6",
        "192.168.64.6",
        "192.168.64.6",
        "192.168.64.6"
      ]
    },
    {
      "name": "ns-lab-ips",
      "id": "id-ns-lab-ips",
      "description": "IP adresses in namespace lab",
      "ranges": [
        "172.17.0.14",
        "